In [1]:
import pandas as pd
import os
#import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from tensorflow.contrib.layers import fully_connected, batch_norm
import math
import numpy as np
import random

In [2]:
df = pd.read_csv('./DL_final_project/DL_Taiwan_data/sinica/201701_Taiwan.csv',
                 usecols=['Date','Time','PM2.5','PM10','PM1','Temperature','Humidity','lon','lat'])
#把201701_Taiwan.csv的header的' lat',' lon'改成'lat','lon' (多了空格)
#len(df)

In [3]:
def getAreaSeriesFromIdx(s):
    #f = s.mask(s['in_area'], s['lon_g']*3+s['lat_g']+1, axis=0)['lat']
    f = s.mask(s['in_area'], s['lat_g']+1, axis=0)['lat']
    f = f.where(s['in_area'], 0)
    return f

In [4]:
#if train_for_taiwan:
#    df = df[(df['lat']>= 21 )& (df['lat'] <= 27)&(df['lon']>=118)&(df['lon']<=122)]


In [11]:
df

,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp,Hour,in_area,lon_g,lat_g,area
0,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
1,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
2,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
3,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
4,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
5,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
6,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
7,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
8,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
9,2017-01-01,08:00:01,70.0,87.0,52.0,23.50,85.0,22.853,120.546,2017-01-01 08:00:01,8,True,1.0,0.0,1.0


In [5]:
root_logdir = "tf_logs"
batch_log_step = 50
early_stopping_epochs = 10

In [14]:
def dnn(area_num, export_dir, X_1, y_1, X_2, y_2):
    tf.reset_default_graph()
    
    print("model will be exported in",export_dir)
    
    # logs
    start_time = datetime.now()
    now = start_time.strftime("%Y%m%d_%H%M%S")
    logdir = "./{}/run-{}".format(root_logdir, now)
    print("now=",now)
    
    # dnn graph defs
    n_input = 5
    n_epochs = 100
    n_hidden = [100,100,10]
    act_fn = tf.nn.sigmoid
    learning_rate = 0.001
    batch_normalization = False
    batch_size = 10000
    #batch_size = 1000 #小範圍測試用
    mult_bias = 1000

    # I/O
    with tf.name_scope("Input"):
        X = tf.placeholder(tf.float32, [None, n_input], name="X")
        is_training = tf.placeholder(tf.bool, shape=(), name="is_training")
    with tf.name_scope("Output"):
        y = tf.placeholder(tf.float32, [None, 1], name="y")
        y_biased = y/mult_bias
    
    # batch norm 
    he_init = tf.contrib.layers.variance_scaling_initializer()
    with tf.name_scope("BatchNormArgs"):
        bn_params = {
            'is_training': is_training,
            'decay': 0.99,
            'updates_collections': None,
            'scale': True
        }
    
    # DNN
    with tf.name_scope("DNN"):
        with tf.contrib.framework.arg_scope(
                [fully_connected],
                weights_initializer = he_init,
                normalizer_fn = batch_norm if batch_normalization else None,
                normalizer_params = bn_params if batch_normalization else None
                ):
            h1=fully_connected(X ,n_hidden[0],activation_fn=act_fn,scope="h1")
            h2=fully_connected(h1,n_hidden[1],activation_fn=act_fn,scope="h2")
            h3=fully_connected(h2,n_hidden[2],activation_fn=act_fn,scope="h3")
        logits=fully_connected(h3, 1, weights_initializer=he_init, activation_fn=act_fn,scope="out")
    
    with tf.name_scope("Cost"):
        #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="cost")
        cost = tf.losses.mean_squared_error(logits, y_biased)
    with tf.name_scope("AdamOptimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        minimizer = optimizer.minimize(cost)
    
    init = tf.global_variables_initializer()
    with tf.name_scope("ModelSaver"):
        saver = tf.train.Saver()
    
    with tf.name_scope("Predict"):
        predict = tf.multiply(logits, mult_bias, name="predict")
        
    with tf.name_scope("Error"):
        ave_of_batch_y = tf.reduce_mean(y_biased)
        error = tf.abs(logits - y_biased)/ave_of_batch_y
        relative_err = tf.reduce_mean(error)
    
    with tf.name_scope("Summaries-Train"):
        cost_summary = tf.summary.scalar('cost_function',cost)
        error_summary = tf.summary.scalar('relative_err',relative_err)
    with tf.name_scope("Summaries-Validation"):
        v_cost_summary = tf.summary.scalar('v_cost_function',cost)
        v_error_summary = tf.summary.scalar('v_relative_err',relative_err)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    with tf.Session() as sess:
        sess.run(init)
        
        total_batch = len(X_1)//batch_size
        #total_batch = validation_idx_start//batch_size
        print("Total batch:",total_batch)
        #X_va, y_va = dfa_X[validation_idx_start:test_idx_start], dfa_Y[validation_idx_start:test_idx_start]
        X_va = X_2
        y_va = y_2
        #X_test = X_3
        #y_test = y_3
        
        best_va_err_triggered = False
        best_va_err = 0
        early_stopping_triggered = False
        stopping_epoch = 0
        step = 0
        save_path = ""
        best_save_path = ""
        
        perm = np.arange(len(X_1))
        np.random.shuffle(perm)
        X_t = X_1[perm]
        y_t = y_1[perm]
        
        for epoch in range(n_epochs):
            for batch in range(total_batch):
                X_ba = X_t[batch*batch_size: (batch+1)*batch_size]
                y_ba = y_t[batch*batch_size: (batch+1)*batch_size]
                _, c = sess.run([minimizer, cost], feed_dict = {is_training: True, X:X_ba, y:y_ba})
                
                if batch % batch_log_step == 0:
                    print("Epoch %4d batch %5d"%(epoch,batch))
                    cost_summary_str = cost_summary.eval(feed_dict={is_training:False, X:X_ba, y:y_ba})
                    step = epoch * total_batch + batch
                    file_writer.add_summary(cost_summary_str, step)
                    error_summary_str = error_summary.eval(feed_dict={is_training:False, X:X_ba, y:y_ba})
                    file_writer.add_summary(error_summary_str, step)
            
            save_path = saver.save(sess, "./checkpoint/model_"+now+".ckpt")
            va_err, va_cost = sess.run([relative_err, cost], feed_dict={is_training:False, X:X_va, y:y_va})
            va_l = sess.run(predict, feed_dict={is_training:False, X:X_va, y:y_va})
            rnd_result_idx = random.randint(0, len(X_va)-1)
            
            print(va_l[rnd_result_idx], y_va[rnd_result_idx]) # print 1st prediction result
            v_error_summary_str = v_error_summary.eval(feed_dict={is_training:False, X:X_va, y:y_va})
            file_writer.add_summary(v_error_summary_str, step)
            v_cost_summary_str = v_cost_summary.eval(feed_dict={is_training:False, X:X_va, y:y_va})
            file_writer.add_summary(v_cost_summary_str, step)
            print("Epoch %4d val.cost %3.6f val.err %3.2f%%"%(epoch,va_cost,va_err*100),end=" ")
            
            if best_va_err_triggered:
                if va_err < best_va_err:
                    print("best")
                    stopping_epoch = 0
                    best_va_err = va_err
                    
                    best_save_path = saver.save(sess, "./best_model/model_"+now)
                else:
                    stopping_epoch += 1
                    print("stopping %3d"%stopping_epoch)
                if stopping_epoch >= early_stopping_epochs:
                    early_stopping_triggered = True
                    print("Early stopping triggered: Step: %10d, val.err %3.2f%%"%(step, va_err*100))
            else:
                best_va_err = va_err
                best_va_err_triggered = True
                print("best")
                best_save_path = saver.save(sess, "./best_model/model_"+now)
            
            if early_stopping_triggered:
                break
        
        finish_time = datetime.now()
        print("best model saved to:", best_save_path)
        file_writer.close()
        elapse_time = finish_time - start_time
        total_seconds = elapse_time.total_seconds()
        print("Total time:", total_seconds)
        
        #X_test = dfa_X[test_idx_start:idx_end]
        #y_test = dfa_Y[test_idx_start:idx_end]
        #saver.restore(sess, "./best_model/model_"+now)
        #best_err = relative_err.eval({is_training: False, X: X_test, y: y_test})
        #print("Test Err: %3.2f%%"%(best_err*100))
        
        builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
        builder.add_meta_graph_and_variables(sess, ["tag"], signature_def_map= {
            "model": tf.saved_model.signature_def_utils.predict_signature_def(
                inputs= {"X": X, "is_training": is_training},
                outputs= {"predict": predict})
        })
        builder.save()
        
        """
        print("Predicting...")
        predict_values = sess.run(predict, feed_dict={is_training: False, X: X_w})
        print("Predict End")
        return predict_values
        """  

In [15]:
# df2: validation
# df3: test
df_v = pd.read_csv('./DL_final_project/DL_Taiwan_data/sinica/201702_Taiwan.csv',
                 usecols=['Date','Time','PM2.5','PM10','PM1','Temperature','Humidity','lon','lat'])
test_idx_start = math.floor(len(df_v)*0.5)
df2 = df_v[:test_idx_start]
df3 = df_v[test_idx_start:]


In [16]:
df = df.assign(Timestamp = pd.to_datetime(df['Date']+' '+df['Time']))
df = df.assign(Hour = lambda x: x['Timestamp'].dt.hour)
df = df.assign(in_area = lambda x:(x['lat']>= 21 )& (x['lat'] <= 27)&(x['lon']>=118)&(x['lon']<=122))
df = df.assign(lon_g = lambda x:(x.lon-118)//2)
df = df.assign(lat_g = lambda x:(x.lat-21)//2)
df = df.assign(area = getAreaSeriesFromIdx)

df2 = df2.assign(Timestamp = pd.to_datetime(df2['Date']+' '+df2['Time']))
df2 = df2.assign(Hour = lambda x: x['Timestamp'].dt.hour)
df2 = df2.assign(in_area = lambda x:(x['lat']>= 21 )& (x['lat'] <= 27)&(x['lon']>=118)&(x['lon']<=122))
df2 = df2.assign(lon_g = lambda x:(x.lon-118)//2)
df2 = df2.assign(lat_g = lambda x:(x.lat-21)//2)
df2 = df2.assign(area = getAreaSeriesFromIdx)

In [12]:
df

,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp,Hour,in_area,lon_g,lat_g,area
0,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
1,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
2,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
3,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
4,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
5,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
6,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
7,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
8,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
9,2017-01-01,08:00:01,70.0,87.0,52.0,23.50,85.0,22.853,120.546,2017-01-01 08:00:01,8,True,1.0,0.0,1.0


In [17]:
area_num = 4
# check folder
export_check_num = 1
export_dir = './final_model/'
while os.path.exists(export_dir):
    export_check_num += 1
    export_dir = './final_model_'+str(export_check_num)+'/'
for a in range(area_num):
    print('area', a)
    
    dfa = df[df['area'] == a]
    df_X = dfa[['Hour','PM10','PM1','Temperature','Humidity']]
    df_X = df_X.values
    df_Y = dfa[['PM2.5']]
    df_Y = df_Y.values
    print('train len =', len(df_X))
    
    df2a = df2[df2['area']==a]
    df2_X = df2a[['Hour','PM10','PM1','Temperature','Humidity']]
    df2_X = df2_X.values
    df2_Y = df2a[['PM2.5']]
    df2_Y = df2_Y.values
    print('test len =',len(df2_X))
    
    
    export_dir_area = export_dir + 'area_'+str(a)+"/"
    dfa.describe(include="all")
    dnn(a, export_dir_area, df_X, df_Y, df2_X, df2_Y)

area 0
train len = 339197
test len = 97343
model will be exported in ./final_model_6/area_0/
now= 20180616_222907
Total batch: 33
Epoch    0 batch     0
[ 185.06806946] [26]
Epoch    0 val.cost 0.026515 val.err 647.43% best
Epoch    1 batch     0
[ 140.38665771] [52]
Epoch    1 val.cost 0.013522 val.err 460.50% best
Epoch    2 batch     0
[ 122.04946899] [15]
Epoch    2 val.cost 0.009543 val.err 385.42% best
Epoch    3 batch     0
[ 107.57241058] [51]
Epoch    3 val.cost 0.007261 val.err 334.79% best
Epoch    4 batch     0
[ 98.98413086] [10]
Epoch    4 val.cost 0.005726 val.err 296.01% best
Epoch    5 batch     0
[ 90.76091003] [46]
Epoch    5 val.cost 0.004654 val.err 265.65% best
Epoch    6 batch     0
[ 84.87249756] [71]
Epoch    6 val.cost 0.003874 val.err 241.17% best
Epoch    7 batch     0
[ 80.20446014] [9]
Epoch    7 val.cost 0.003281 val.err 220.76% best
Epoch    8 batch     0
[ 75.63459778] [25]
Epoch    8 val.cost 0.002815 val.err 203.29% best
Epoch    9 batch     0
[ 71.65

Epoch   88 batch     0
[ 28.14460754] [50]
Epoch   88 val.cost 0.000264 val.err 53.24% best
Epoch   89 batch     0
[ 28.12389565] [43]
Epoch   89 val.cost 0.000264 val.err 53.21% best
Epoch   90 batch     0
[ 28.07658577] [18]
Epoch   90 val.cost 0.000264 val.err 53.18% best
Epoch   91 batch     0
[ 28.07400703] [39]
Epoch   91 val.cost 0.000264 val.err 53.15% best
Epoch   92 batch     0
[ 28.03269577] [18]
Epoch   92 val.cost 0.000264 val.err 53.12% best
Epoch   93 batch     0
[ 28.0056591] [15]
Epoch   93 val.cost 0.000264 val.err 53.10% best
Epoch   94 batch     0
[ 27.98855972] [15]
Epoch   94 val.cost 0.000264 val.err 53.08% best
Epoch   95 batch     0
[ 27.97222137] [10]
Epoch   95 val.cost 0.000264 val.err 53.06% best
Epoch   96 batch     0
[ 27.9527607] [9]
Epoch   96 val.cost 0.000263 val.err 53.04% best
Epoch   97 batch     0
[ 27.95249367] [24]
Epoch   97 val.cost 0.000263 val.err 53.03% best
Epoch   98 batch     0
[ 27.95372581] [46]
Epoch   98 val.cost 0.000263 val.err 53.

[ 52.8619194] [38]
Epoch   58 val.cost 0.000360 val.err 26.97% best
Epoch   59 batch     0
Epoch   59 batch    50
[ 52.86167908] [59]
Epoch   59 val.cost 0.000360 val.err 26.97% best
Epoch   60 batch     0
Epoch   60 batch    50
[ 52.86076736] [55]
Epoch   60 val.cost 0.000360 val.err 26.97% best
Epoch   61 batch     0
Epoch   61 batch    50
[ 52.85978317] [43]
Epoch   61 val.cost 0.000360 val.err 26.97% best
Epoch   62 batch     0
Epoch   62 batch    50
[ 52.86294174] [98]
Epoch   62 val.cost 0.000360 val.err 26.97% best
Epoch   63 batch     0
Epoch   63 batch    50
[ 52.86727524] [33]
Epoch   63 val.cost 0.000360 val.err 26.97% best
Epoch   64 batch     0
Epoch   64 batch    50
[ 52.86222839] [72]
Epoch   64 val.cost 0.000360 val.err 26.97% best
Epoch   65 batch     0
Epoch   65 batch    50
[ 52.86272049] [60]
Epoch   65 val.cost 0.000360 val.err 26.97% best
Epoch   66 batch     0
Epoch   66 batch    50
[ 52.86099625] [50]
Epoch   66 val.cost 0.000360 val.err 26.97% best
Epoch   67 b

Epoch   16 batch   200
[ 24.41350746] [7]
Epoch   16 val.cost 0.000143 val.err 14.51% best
Epoch   17 batch     0
Epoch   17 batch    50
Epoch   17 batch   100
Epoch   17 batch   150
Epoch   17 batch   200
[ 24.12193489] [35]
Epoch   17 val.cost 0.000141 val.err 13.84% best
Epoch   18 batch     0
Epoch   18 batch    50
Epoch   18 batch   100
Epoch   18 batch   150
Epoch   18 batch   200
[ 23.82143021] [14]
Epoch   18 val.cost 0.000140 val.err 13.41% best
Epoch   19 batch     0
Epoch   19 batch    50
Epoch   19 batch   100
Epoch   19 batch   150
Epoch   19 batch   200
[ 34.75971222] [31]
Epoch   19 val.cost 0.000134 val.err 13.12% best
Epoch   20 batch     0
Epoch   20 batch    50
Epoch   20 batch   100
Epoch   20 batch   150
Epoch   20 batch   200
[ 23.57168198] [53]
Epoch   20 val.cost 0.000133 val.err 12.94% best
Epoch   21 batch     0
Epoch   21 batch    50
Epoch   21 batch   100
Epoch   21 batch   150
Epoch   21 batch   200
[ 38.41492081] [38]
Epoch   21 val.cost 0.000132 val.err 1

Epoch   60 val.cost 0.000073 val.err 7.64% best
Epoch   61 batch     0
Epoch   61 batch    50
Epoch   61 batch   100
Epoch   61 batch   150
Epoch   61 batch   200
[ 34.80601501] [34]
Epoch   61 val.cost 0.000073 val.err 7.58% best
Epoch   62 batch     0
Epoch   62 batch    50
Epoch   62 batch   100
Epoch   62 batch   150
Epoch   62 batch   200
[ 40.94760513] [39]
Epoch   62 val.cost 0.000073 val.err 7.60% stopping   1
Epoch   63 batch     0
Epoch   63 batch    50
Epoch   63 batch   100
Epoch   63 batch   150
Epoch   63 batch   200
[ 34.10463333] [42]
Epoch   63 val.cost 0.000073 val.err 7.57% best
Epoch   64 batch     0
Epoch   64 batch    50
Epoch   64 batch   100
Epoch   64 batch   150
Epoch   64 batch   200
[ 12.5003376] [13]
Epoch   64 val.cost 0.000074 val.err 7.55% best
Epoch   65 batch     0
Epoch   65 batch    50
Epoch   65 batch   100
Epoch   65 batch   150
Epoch   65 batch   200
[ 60.14717484] [61]
Epoch   65 val.cost 0.000073 val.err 7.53% best
Epoch   66 batch     0
Epoch  

Epoch    5 val.cost 0.000861 val.err 55.76% best
Epoch    6 batch     0
Epoch    6 batch    50
Epoch    6 batch   100
[ 41.27970505] [9]
Epoch    6 val.cost 0.000780 val.err 50.97% best
Epoch    7 batch     0
Epoch    7 batch    50
Epoch    7 batch   100
[ 38.80788422] [42]
Epoch    7 val.cost 0.000729 val.err 48.00% best
Epoch    8 batch     0
Epoch    8 batch    50
Epoch    8 batch   100
[ 36.78100967] [30]
Epoch    8 val.cost 0.000694 val.err 45.97% best
Epoch    9 batch     0
Epoch    9 batch    50
Epoch    9 batch   100
[ 35.01211166] [39]
Epoch    9 val.cost 0.000672 val.err 44.62% best
Epoch   10 batch     0
Epoch   10 batch    50
Epoch   10 batch   100
[ 33.6419487] [29]
Epoch   10 val.cost 0.000657 val.err 43.74% best
Epoch   11 batch     0
Epoch   11 batch    50
Epoch   11 batch   100
[ 32.4530983] [14]
Epoch   11 val.cost 0.000647 val.err 43.11% best
Epoch   12 batch     0
Epoch   12 batch    50
Epoch   12 batch   100
[ 41.93791962] [45]
Epoch   12 val.cost 0.000640 val.err 

Epoch   65 batch     0
Epoch   65 batch    50
Epoch   65 batch   100
[ 45.57587433] [42]
Epoch   65 val.cost 0.000613 val.err 38.46% best
Epoch   66 batch     0
Epoch   66 batch    50
Epoch   66 batch   100
[ 57.86778641] [56]
Epoch   66 val.cost 0.000612 val.err 38.36% best
Epoch   67 batch     0
Epoch   67 batch    50
Epoch   67 batch   100
[ 27.94246864] [17]
Epoch   67 val.cost 0.000612 val.err 38.33% best
Epoch   68 batch     0
Epoch   68 batch    50
Epoch   68 batch   100
[ 25.75528145] [7]
Epoch   68 val.cost 0.000611 val.err 38.14% best
Epoch   69 batch     0
Epoch   69 batch    50
Epoch   69 batch   100
[ 29.91609001] [17]
Epoch   69 val.cost 0.000611 val.err 38.08% best
Epoch   70 batch     0
Epoch   70 batch    50
Epoch   70 batch   100
[ 26.25819969] [13]
Epoch   70 val.cost 0.000611 val.err 37.92% best
Epoch   71 batch     0
Epoch   71 batch    50
Epoch   71 batch   100
[ 26.54151726] [44]
Epoch   71 val.cost 0.000610 val.err 37.84% best
Epoch   72 batch     0
Epoch   72 b

In [15]:
df3

,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon
1384569,2017-02-13,04:05:37,56,74,40,18.62,92,22.608,120.271
1384570,2017-02-13,04:05:38,29,31,21,17.12,71,24.252,120.739
1384571,2017-02-13,04:05:38,49,62,35,20.50,81,22.514,120.395
1384572,2017-02-13,04:05:39,17,20,12,7.25,82,24.255,121.259
1384573,2017-02-13,04:05:40,37,43,26,19.12,65,22.993,120.213
1384574,2017-02-13,04:05:41,37,47,27,18.00,73,23.189,120.192
1384575,2017-02-13,04:05:41,53,70,38,20.12,71,22.622,120.286
1384576,2017-02-13,04:05:41,36,41,26,17.50,75,23.018,120.134
1384577,2017-02-13,04:05:42,54,71,38,18.62,82,22.708,120.438
1384578,2017-02-13,04:05:42,24,26,19,21.00,51,24.175,120.716


In [18]:
df3 = df3.assign(Timestamp = pd.to_datetime(df3['Date']+' '+df3['Time']))
df3 = df3.assign(Hour = lambda x: x['Timestamp'].dt.hour)
df3 = df3.assign(in_area = lambda x:(x['lat']>= 21 )& (x['lat'] <= 27)&(x['lon']>=118)&(x['lon']<=122))
df3 = df3.assign(lon_g = lambda x:(x.lon-118)//2)
df3 = df3.assign(lat_g = lambda x:(x.lat-21)//2)
df3 = df3.assign(area = getAreaSeriesFromIdx)
df3 = df3.reset_index(drop=True)

In [27]:
#[i for i in range (len(df3))]
df3 = df3.assign(idx = df3.index)
df3

,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp,Hour,in_area,lon_g,lat_g,area,idx
0,2017-02-13,04:05:37,56,74,40,18.62,92,22.608,120.271,2017-02-13 04:05:37,4,True,1.0,0.0,1.0,0
1,2017-02-13,04:05:38,29,31,21,17.12,71,24.252,120.739,2017-02-13 04:05:38,4,True,1.0,1.0,2.0,1
2,2017-02-13,04:05:38,49,62,35,20.50,81,22.514,120.395,2017-02-13 04:05:38,4,True,1.0,0.0,1.0,2
3,2017-02-13,04:05:39,17,20,12,7.25,82,24.255,121.259,2017-02-13 04:05:39,4,True,1.0,1.0,2.0,3
4,2017-02-13,04:05:40,37,43,26,19.12,65,22.993,120.213,2017-02-13 04:05:40,4,True,1.0,0.0,1.0,4
5,2017-02-13,04:05:41,37,47,27,18.00,73,23.189,120.192,2017-02-13 04:05:41,4,True,1.0,1.0,2.0,5
6,2017-02-13,04:05:41,53,70,38,20.12,71,22.622,120.286,2017-02-13 04:05:41,4,True,1.0,0.0,1.0,6
7,2017-02-13,04:05:41,36,41,26,17.50,75,23.018,120.134,2017-02-13 04:05:41,4,True,1.0,1.0,2.0,7
8,2017-02-13,04:05:42,54,71,38,18.62,82,22.708,120.438,2017-02-13 04:05:42,4,True,1.0,0.0,1.0,8
9,2017-02-13,04:05:42,24,26,19,21.00,51,24.175,120.716,2017-02-13 04:05:42,4,True,1.0,1.0,2.0,9


In [41]:
round_to_int = False
result_pd = pd.DataFrame({'PM2.5':[],'idx':[]})
for a in range(area_num):
#if True:
    #a = 0
    print('Area', a)
    tf.reset_default_graph()
    dft = df3[df3['area']==a]
    predict_values = np.zeros(len(dft))
    print(len(dft))
    df_X = dft[['Hour','PM10','PM1','Temperature','Humidity','lon','lat']]
    df_X = df_X.values
    df_Y = dft[['PM2.5']]
    df_Y = df_Y.values
    
    batch_size = 10000
    disp_log = 10
    n_batches = math.ceil(len(dft) / batch_size)
    with tf.Session(graph=tf.Graph()) as sess:
        tf.saved_model.loader.load(sess, ["tag"], export_dir + 'area_'+str(a)+"/")
        graph = tf.get_default_graph()
        X = graph.get_tensor_by_name("Input/X:0")
        is_training = graph.get_tensor_by_name("Input/is_training:0")
        predict = graph.get_tensor_by_name("Predict/predict:0")

        for i in range(n_batches):
            if i % disp_log == 0:
                print('batch',i)
            if i == n_batches - 1:
                X_batch = df_X[i*batch_size:]
                predict_batch = sess.run(predict, {is_training: False, X: X_batch})
                predict_values[i*batch_size:] = predict_batch[:,0]
            else:
                X_batch = df_X[i*batch_size:(i+1)*batch_size]
                predict_batch = sess.run(predict, {is_training: False, X: X_batch})
                predict_values[i*batch_size:(i+1)*batch_size] = predict_batch[:,0]


        if round_to_int:
            predict_values = [int(v) for v in predict_values]
    
    area_result = pd.DataFrame({'PM2.5':predict_values, 'idx':dft['idx']})
    result_pd = result_pd.append(area_result)
print('end')

Area 0
117992
INFO:tensorflow:Restoring parameters from b'./final_model_5/area_0/variables\\variables'
batch 0
batch 10
Area 1
244601
INFO:tensorflow:Restoring parameters from b'./final_model_5/area_1/variables\\variables'
batch 0
batch 10
batch 20
Area 2
723014
INFO:tensorflow:Restoring parameters from b'./final_model_5/area_2/variables\\variables'
batch 0
batch 10
batch 20
batch 30
batch 40
batch 50
batch 60
batch 70
Area 3
298963
INFO:tensorflow:Restoring parameters from b'./final_model_5/area_3/variables\\variables'
batch 0
batch 10
batch 20
end


In [43]:
result_pd.sort_values('idx')

,PM2.5,idx
0,57.822571,0.0
1,27.380884,1.0
2,49.965599,2.0
3,15.638891,3.0
4,35.877102,4.0
5,38.026768,5.0
6,54.426346,6.0
7,35.131573,7.0
8,55.201263,8.0
9,23.718853,9.0
